# Hi Kagglers !

Original notebook : https://www.kaggle.com/code/nealart/simple-ensemble-of-public-best-kernels-v-2-2-0?scriptVersionId=92637719

---

Here is a simple ensemble method for submissions.
This idea can be easily referenced from previous competitions

If you think this kernel is good, please upvote the following people who provided the original kernel instead.

*  https://www.kaggle.com/aikhmelnytskyy/happywhale-arcface-baseline-eff7-tpu-768-inference

* https://www.kaggle.com/nghiahoangtrung/0-720-eff-b5-640-rotate

* https://www.kaggle.com/aikhmelnytskyy/happywhale-effnet-b7-fork-with-detic-training

* https://www.kaggle.com/andrej0marinchenko/happywhale-0-679

I added my own result training (best82) on fullbody annotation dataset with beluga only
and was able to achieve significant lb boost. 

I use the pytorch notebook by clemchris to train my model with some adjustment such as fine
tuning the margin 


https://www.kaggle.com/code/clemchris/pytorch-backfin-convnext-arcface

In [1]:
import csv
import pandas as pd 

sub_files = [
                 '../input/best82/submission.csv',
                 '../input/convnext-largebase-tff-b5b6b7/embd_convnext_large_384_in22ft1k_384convnext_base_384_in22ft1k_512tf_efficientnet_b7_ns_512tf_efficientnet_b5_ns_512tf_efficientnet_b6_ns_512_norm_submission.csv',
                 # '../input/convnext-large384-effb7-effb6-effb5-512/embd_convnext_large_384_in22ft1k_384tf_efficientnet_b7_ns_512tf_efficientnet_b5_ns_512tf_efficientnet_b6_ns_512_norm_submission.csv',
                 '../input/simple-ensemble-of-public-best-kernels/submission.csv',
                 '../input/dolphin/effnetb7_effnetv2l_ensemble.csv',
#                  '../input/swin-tranform-submission/submission.csv',#swint
#                  '../input/happywhale-arcface-baseline-eff7-tpu-768-inference/submission.csv',#eff7
#                  '../input/0-720-eff-b5-640-rotate/submission.csv',  #effb5
#                  '../input/happywhale-effnet-b7-fork-with-detic-training/submission.csv', # eff7, 0.699
#                  '../input/simple-vanilla-pytorch-inference-gem-pooling/submission.csv', # 0.694
                 # '../input/arcfaceeffb6inferbaseline/submission.csv',
                 # '../input/happywhale-effnet-b7-fork-with-detic-crop/submission.csv',
                 
]

# Weights of the individual subs
# sub_weight = [
#                 0.5,
# #                 0.2,
# #                 0.2,
# #                 0.1,
#                 0.5,
# ]
#             ]
# Weights of the individual subs
N_wight=0.5
Base_wight=1.5

l1 = [0.782, 0.778, 0.777, 0.766]
l2 = [Base_wight+N_wight*i for i in range(len(l1))]
sub_weight = list(map(lambda x,y: x**y ,l1,l2))


In [2]:
Hlabel = 'image' 
Htarget = 'predictions'
npt = 6
place_weights = {}
for i in range(npt):
    place_weights[i] = (1 / (i + 1))

print(place_weights)

lg = len(sub_files)
sub = [None]*lg
for i, file in enumerate( sub_files ):   
    print("Reading {}: w={} - {}". format(i, sub_weight[i], file))
    reader = csv.DictReader(open(file,"r"))
    sub[i] = sorted(reader, key=lambda d: str(d[Hlabel]))

out = open("submission.csv", "w", newline='')
writer = csv.writer(out)
writer.writerow([Hlabel,Htarget])

for p, row in enumerate(sub[0]):
    target_weight = {}
    for s in range(lg):
        row1 = sub[s][p]
        for ind, trgt in enumerate(row1[Htarget].split(' ')):
            target_weight[trgt] = target_weight.get(trgt,0) + (place_weights[ind]*sub_weight[s])
    tops_trgt = sorted(target_weight, key=target_weight.get, reverse=True)[:npt]
    writer.writerow([row1[Hlabel], " ".join(tops_trgt)])
out.close()

{0: 1.0, 1: 0.5, 2: 0.3333333333333333, 3: 0.25, 4: 0.2, 5: 0.16666666666666666}
Reading 0: w=0.6915285735239001 - ../input/best82/submission.csv
Reading 1: w=0.605284 - ../input/convnext-largebase-tff-b5b6b7/embd_convnext_large_384_in22ft1k_384convnext_base_384_in22ft1k_512tf_efficientnet_b7_ns_512tf_efficientnet_b5_ns_512tf_efficientnet_b6_ns_512_norm_submission.csv
Reading 2: w=0.5321726450388605 - ../input/simple-ensemble-of-public-best-kernels/submission.csv
Reading 3: w=0.449455096 - ../input/dolphin/effnetb7_effnetv2l_ensemble.csv


<center>
    <h1 style='color: #f13658'> Thanks for reading 👍 <h1>